In [1]:
%scala
import org.apache.spark.ml.linalg.Vectors
val denseVec = Vectors.dense(1.0, 2.0, 3.0)
val size = 3
val idx = Array(1,2) // locations in vector
val values = Array(2.0,3.0)
val sparseVec = Vectors.sparse(size, idx, values)
sparseVec.toDense
denseVec.toSparse


In [2]:

from pyspark.ml.linalg import Vectors
denseVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2] # locations in vector
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)
# sparseVec.toDense() # these two don't work, not sure why
# denseVec.toSparse() # will debug later

In [4]:
%scala
var df = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")

In [5]:
df = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")
df.orderBy("value2").show()

In [6]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/t4wp8ci61497648354438
ls

In [7]:
%scala
val libsvmData = spark.read.format("libsvm")
.load("FileStore/tables/t4wp8ci61497648354438/sample_libsvm_data.txt")

In [8]:
libsvmData = spark.read.format("libsvm")\
.load("FileStore/tables/t4wp8ci61497648354438/sample_libsvm_data.txt")

In [9]:
%scala
import org.apache.spark.ml.feature.RFormula
val supervised = new RFormula()
.setFormula("lab ~ . + color:value1 + color:value2")

In [10]:
from pyspark.ml.feature import RFormula
supervised = RFormula()\
.setFormula("lab ~ . + color:value1 + color:value2")

In [11]:
%scala
val fittedRF = supervised.fit(df)
val preparedDF = fittedRF.transform(df)

In [12]:
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show()

In [13]:
%scala
val Array(train, test) = preparedDF.randomSplit(Array(0.7, 0.3))

In [14]:
train, test = preparedDF.randomSplit([0.7, 0.3])

In [15]:
%scala
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression()
.setLabelCol("label")
.setFeaturesCol("features")

In [16]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()\
.setLabelCol("label")\
.setFeaturesCol("features")

In [17]:
%scala
val fittedLR = lr.fit(train)

In [18]:
fittedLR = lr.fit(train)

In [19]:
fittedLR.transform(train).select("label", "prediction").show()

In [20]:
%scala
val Array(train, test) = df.randomSplit(Array(0.7, 0.3))

In [21]:
train, test = df.randomSplit([0.7, 0.3])

In [22]:
%scala
val rForm = new RFormula()
val lr = new LogisticRegression()
.setLabelCol("label")
.setFeaturesCol("features")


In [23]:
rForm = RFormula()

lr = LogisticRegression()\
.setLabelCol("label")\
.setFeaturesCol("features")

In [24]:
%scala
import org.apache.spark.ml.Pipeline
val stages = Array(rForm, lr)
val pipeline = new Pipeline().setStages(stages)

In [25]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

In [26]:
%scala
import org.apache.spark.ml.tuning.ParamGridBuilder
val params = new ParamGridBuilder()
.addGrid(rForm.formula, Array(
"lab ~ . + color:value1",
"lab ~ . + color:value1 + color:value2"))
.addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
.addGrid(lr.regParam, Array(0.1, 2.0))
.build()

In [27]:
from pyspark.ml.tuning import ParamGridBuilder
params = ParamGridBuilder()\
.addGrid(rForm.formula, [
"lab ~ . + color:value1",
"lab ~ . + color:value1 + color:value2"])\
.addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
.addGrid(lr.regParam, [0.1, 2.0])\
.build()

In [28]:
%scala
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()
.setMetricName("areaUnderROC")
.setRawPredictionCol("prediction")
.setLabelCol("label")

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
.setMetricName("areaUnderROC")\
.setRawPredictionCol("prediction")\
.setLabelCol("label")

In [30]:
%scala
import org.apache.spark.ml.tuning.TrainValidationSplit
val tvs = new TrainValidationSplit()
.setTrainRatio(0.75) // also the default.
.setEstimatorParamMaps(params)
.setEstimator(pipeline)
.setEvaluator(evaluator)

In [31]:
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
.setTrainRatio(0.75)\
.setEstimatorParamMaps(params)\
.setEstimator(pipeline)\
.setEvaluator(evaluator)

In [32]:
%scala
val tvsFitted = tvs.fit(train)

In [33]:
tvsFitted = tvs.fit(train)

In [34]:
evaluator.evaluate(tvsFitted.transform(test))

In [35]:
%scala
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.LogisticRegressionModel
val trainedPipeline = tvsFitted.bestModel.asInstanceOf[PipelineModel]
val TrainedLR = trainedPipeline.stages(1)
.asInstanceOf[LogisticRegressionModel]
val summaryLR = TrainedLR.summary
summaryLR.objectiveHistory

In [36]:
%scala
tvsFitted.write.overwrite().save("/tmp/modelLocation")

In [37]:
%scala
import org.apache.spark.ml.tuning.TrainValidationSplitModel
val model = TrainValidationSplitModel.load("/tmp/modelLocation")
model.transform(test)

In [38]:
%scala
val sales = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")
.coalesce(5)
.where("Description IS NOT NULL")
val fakeIntDF = spark.read.parquet("/FileStore/tables/w90rmm721497649161288/part_00000_ce2a44c8_feb4_4369_a2c3_4bf2f0e63b07_c000_gz-d65b9.parquet")
var simpleDF = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")
val scaleDF = spark.read.parquet("/FileStore/tables/48se0cwp1497649233518/part_00000_cd03406a_cc9b_42b0_9299_1e259fdd9382_c000_gz-2e0be.parquet")

In [39]:
sales = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")\
.coalesce(5)\
.where("Description IS NOT NULL")
fakeIntDF = spark.read.parquet("/FileStore/tables/w90rmm721497649161288/part_00000_ce2a44c8_feb4_4369_a2c3_4bf2f0e63b07_c000_gz-d65b9.parquet")
simpleDF = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")
scaleDF = spark.read.parquet("/FileStore/tables/48se0cwp1497649233518/part_00000_cd03406a_cc9b_42b0_9299_1e259fdd9382_c000_gz-2e0be.parquet")
sales.cache()

In [40]:
%scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer().setInputCol("Description")
tkn.transform(sales).show()

In [41]:
%scala
import org.apache.spark.ml.feature.StandardScaler
val ss = new StandardScaler().setInputCol("features")
ss.fit(scaleDF).transform(scaleDF).show(false)

In [42]:
%scala
import org.apache.spark.ml.feature.RFormula
val supervised = new RFormula()
.setFormula("lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

In [43]:
from pyspark.ml.feature import RFormula
supervised = RFormula()\
.setFormula("lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

In [44]:
%scala
import org.apache.spark.ml.feature.SQLTransformer
val basicTransformation = new SQLTransformer()
.setStatement("""
SELECT sum(Quantity), count(*), CustomerID
FROM __THIS__
GROUP BY CustomerID
""")
basicTransformation.transform(sales).show()

In [45]:
from pyspark.ml.feature import SQLTransformer
basicTransformation = SQLTransformer()\
.setStatement("""
SELECT sum(Quantity), count(*), CustomerID
FROM __THIS__
GROUP BY CustomerID
""")
basicTransformation.transform(sales).show()

In [46]:
%scala
import org.apache.spark.ml.feature.VectorAssembler
val va = new VectorAssembler()
.setInputCols(Array("int1", "int2", "int3"))
va.transform(fakeIntDF).show()

In [47]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

In [48]:
%scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer()
.setInputCol("Description")
.setOutputCol("DescriptionOut")
val tokenized = tkn.transform(sales)
tokenized.show()

In [49]:
from pyspark.ml.feature import Tokenizer
tkn = Tokenizer()\
.setInputCol("Description")\
.setOutputCol("DescriptionOut")
tokenized = tkn.transform(sales)
tokenized.show()

In [50]:
%scala
import org.apache.spark.ml.feature.RegexTokenizer
val rt = new RegexTokenizer()
.setInputCol("Description")
.setOutputCol("DescriptionOut")
.setPattern(" ") // starting simple
.setToLowercase(true)
rt.transform(sales).show()

In [51]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescriptionOut")\
.setPattern(" ")\
.setToLowercase(True)
rt.transform(sales).show()

In [52]:
%scala
import org.apache.spark.ml.feature.StopWordsRemover
val englishStopWords = StopWordsRemover
.loadDefaultStopWords("english")
val stops = new StopWordsRemover()
.setStopWords(englishStopWords)
.setInputCol("DescriptionOut")
stops.transform(tokenized).show()

In [53]:
from pyspark.ml.feature import StopWordsRemover
englishStopWords = StopWordsRemover\
.loadDefaultStopWords("english")
stops = StopWordsRemover()\
.setStopWords(englishStopWords)\
.setInputCol("DescriptionOut")
stops.transform(tokenized).show()

In [54]:
%scala
import org.apache.spark.ml.feature.NGram
val unigram = new NGram()
.setInputCol("DescriptionOut")
.setN(1)
val bigram = new NGram()
.setInputCol("DescriptionOut")
.setN(2)
unigram.transform(tokenized).show()
bigram.transform(tokenized).show()

In [55]:
%scala
import org.apache.spark.ml.feature.CountVectorizer
val cv = new CountVectorizer()
.setInputCol("DescriptionOut")
.setOutputCol("countVec")
.setVocabSize(500)
.setMinTF(1)
.setMinDF(2)
val fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show()

In [56]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer()\
.setInputCol("DescriptionOut")\
.setOutputCol("countVec")\
.setVocabSize(500)\
.setMinTF(1)\
.setMinDF(2)
fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show()

In [57]:
%scala
val tfIdfIn = tokenized
.where("array_contains(DescriptionOut, 'red')")
.select("DescriptionOut")
.limit(10)
tfIdfIn.show(false)

In [58]:
tfIdfIn = tokenized\
.where("array_contains(DescriptionOut, 'red')")\
.select("DescriptionOut")\
.limit(10)
tfIdfIn.show(10, False)

In [59]:
%scala
import org.apache.spark.ml.feature.{HashingTF, IDF}
val tf = new HashingTF()
.setInputCol("DescriptionOut")
.setOutputCol("TFOut")
.setNumFeatures(10000)
val idf = new IDF()
.setInputCol("TFOut")
.setOutputCol("IDFOut")
.setMinDocFreq(2)

In [60]:
from pyspark.ml.feature import HashingTF, IDF
tf = HashingTF()\
.setInputCol("DescriptionOut")\
.setOutputCol("TFOut")\
.setNumFeatures(10000)
idf = IDF()\
.setInputCol("TFOut")\
.setOutputCol("IDFOut")\
.setMinDocFreq(2)

In [61]:
%scala
idf.fit(tf.transform(tfIdfIn))
.transform(tf.transform(tfIdfIn))
.show(false)

In [62]:
idf.fit(tf.transform(tfIdfIn))\
.transform(tf.transform(tfIdfIn))\
.show(10, False)

In [63]:
%scala
val contDF = spark.range(500)
.selectExpr("cast(id as double)")

In [64]:
contDF = spark.range(500)\
.selectExpr("cast(id as double)")

In [65]:
%scala
import org.apache.spark.ml.feature.Bucketizer
val bucketBorders = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
val bucketer = new Bucketizer()
.setSplits(bucketBorders)
.setInputCol("id")
bucketer.transform(contDF).show()

In [66]:
from pyspark.ml.feature import Bucketizer
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer()\
.setSplits(bucketBorders)\
.setInputCol("id")
bucketer.transform(contDF).show()

In [67]:
%scala
import org.apache.spark.ml.feature.QuantileDiscretizer
val bucketer = new QuantileDiscretizer()
.setNumBuckets(5)
.setInputCol("id")
val fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

In [68]:
from pyspark.ml.feature import QuantileDiscretizer
bucketer = QuantileDiscretizer()\
.setNumBuckets(5)\
.setInputCol("id")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

In [69]:
%scala
import org.apache.spark.ml.feature.Normalizer
val taxicab = new Normalizer()
.setP(1)
.setInputCol("features")
taxicab.transform(scaleDF).show(false)

In [70]:
from pyspark.ml.feature import Normalizer
taxicab = Normalizer()\
.setP(1)\
.setInputCol("features")
taxicab.transform(scaleDF).show()

In [71]:
%scala
import org.apache.spark.ml.feature.StandardScaler
val sScaler = new StandardScaler()
.setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show(false)

In [72]:
%scala
import org.apache.spark.ml.feature.MinMaxScaler
val minMax = new MinMaxScaler()
.setMin(5)
.setMax(10)
.setInputCol("features")
val fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show(false)

In [73]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler()\
.setMin(5)\
.setMax(10)\
.setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

In [74]:
%scala
import org.apache.spark.ml.feature.MaxAbsScaler
val maScaler = new MaxAbsScaler()
.setInputCol("features")
val fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show(false)

In [75]:
%scala
import org.apache.spark.ml.feature.ElementwiseProduct
import org.apache.spark.ml.linalg.Vectors
val scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
val scalingUp = new ElementwiseProduct()
.setScalingVec(scaleUpVec)
.setInputCol("features")
scalingUp.transform(scaleDF).show()

In [76]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct()\
.setScalingVec(scaleUpVec)\
.setInputCol("features")
scalingUp.transform(scaleDF).show()

In [77]:
%scala
import org.apache.spark.ml.feature.StringIndexer
val labelIndexer = new StringIndexer()
.setInputCol("lab")
.setOutputCol("labelInd")
val idxRes = labelIndexer.fit(simpleDF).transform(simpleDF)
idxRes.show()

In [78]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer()\
.setInputCol("lab")\
.setOutputCol("labelInd")
idxRes = labelIndexer.fit(simpleDF).transform(simpleDF)
idxRes.show()

In [79]:
%scala
val valIndexer = new StringIndexer()
.setInputCol("value1")
.setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

In [80]:
valIndexer = StringIndexer()\
.setInputCol("value1")\
.setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

In [81]:
%scala
valIndexer.setHandleInvalid("skip")
valIndexer.fit(simpleDF).setHandleInvalid("skip")

In [82]:
%scala
import org.apache.spark.ml.feature.IndexToString
val labelReverse = new IndexToString()
.setInputCol("labelInd")
labelReverse.transform(idxRes).show()

In [83]:
from pyspark.ml.feature import IndexToString
labelReverse = IndexToString()\
.setInputCol("labelInd")
labelReverse.transform(idxRes).show()

In [84]:
%scala
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.linalg.Vectors
val idxIn = spark.createDataFrame(Seq(
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)
)).toDF("features", "label")
val indxr = new VectorIndexer()
.setInputCol("features")
.setOutputCol("idxed")
.setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show

In [85]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
idxIn = spark.createDataFrame([
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)
]).toDF("features", "label")
indxr = VectorIndexer()\
.setInputCol("features")\
.setOutputCol("idxed")\
.setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show

In [86]:
%scala
val labelIndexer = new StringIndexer()
.setInputCol("color")
.setOutputCol("colorInd")
val colorLab = labelIndexer.fit(simpleDF).transform(simpleDF)

In [87]:
labelIndexer = StringIndexer()\
.setInputCol("color")\
.setOutputCol("colorInd")
colorLab = labelIndexer.fit(simpleDF).transform(simpleDF)

In [88]:
%scala
import org.apache.spark.ml.feature.OneHotEncoder
val ohe = new OneHotEncoder()
.setInputCol("colorInd")
ohe.transform(colorLab).show()

In [89]:
from pyspark.ml.feature import OneHotEncoder
ohe = OneHotEncoder()\
.setInputCol("colorInd")
ohe.transform(colorLab).show()

In [90]:
%scala
import org.apache.spark.ml.feature.PCA
val pca = new PCA()
.setInputCol("features")
.setK(2)
pca.fit(scaleDF).transform(scaleDF).show(false)

In [91]:
from pyspark.ml.feature import PCA
pca = PCA()\
.setInputCol("features")\
.setK(2)
pca.fit(scaleDF).transform(scaleDF).show()

In [92]:
%scala
import org.apache.spark.ml.feature.PolynomialExpansion
val pe = new PolynomialExpansion()
.setInputCol("features")
.setDegree(2)
pe.transform(scaleDF).show(false)

In [93]:
from pyspark.ml.feature import PolynomialExpansion
pe = PolynomialExpansion()\
.setInputCol("features")\
.setDegree(2)
pe.transform(scaleDF).show()

In [94]:
%scala
import org.apache.spark.ml.feature.ChiSqSelector
val prechi = fittedCV.transform(tokenized)
.where("CustomerId IS NOT NULL")
val chisq = new ChiSqSelector()
.setFeaturesCol("countVec")
.setLabelCol("CustomerID")
.setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi).show()

In [95]:
from pyspark.ml.feature import ChiSqSelector
prechi = fittedCV.transform(tokenized)\
.where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
.setFeaturesCol("countVec")\
.setLabelCol("CustomerID")\
.setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi).show()

In [96]:
%scala
val fittedPCA = pca.fit(scaleDF)
fittedPCA.write.overwrite().save("/tmp/fittedPCA")

In [97]:
%scala
val loadedPCA = PCA.load("/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

In [98]:
%scala
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.types.{ArrayType, StringType, DataType}
import org.apache.spark.ml.param.{IntParam, ParamValidators}
class MyTokenizer(override val uid: String)
extends UnaryTransformer[String, Seq[String], MyTokenizer] with DefaultParamsWritable {
def this() = this(Identifiable.randomUID("myTokenizer"))
val maxWords: IntParam = new IntParam(this, "maxWords", "The max number of words to return.",
ParamValidators.gtEq(0))
def setMaxWords(value: Int): this.type = set(maxWords, value)
def getMaxWords: Integer = $(maxWords)
override protected def createTransformFunc: String => Seq[String] = (inputString: String) => {
inputString.split("\\s").take($(maxWords))
}
override protected def validateInputType(inputType: DataType): Unit = {
require(inputType == StringType, s"Bad input type: $inputType. Requires String.")
}
override protected def outputDataType: DataType = new ArrayType(StringType, true)
}
// this will allow you to read it back in by using this object.
object MyTokenizer extends DefaultParamsReadable[MyTokenizer]
val myT = new MyTokenizer()
.setInputCol("someCol")
.setMaxWords(2)
display(myT.transform(Seq("hello world. This text won't show.").toDF("someCol")))
myT.write.overwrite().save("/tmp/something")

In [99]:
%scala
val sales = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")
.coalesce(5)
.where("Description IS NOT NULL")
val fakeIntDF = spark.read.parquet("/FileStore/tables/w90rmm721497649161288/part_00000_ce2a44c8_feb4_4369_a2c3_4bf2f0e63b07_c000_gz-d65b9.parquet")
var simpleDF = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")
val scaleDF = spark.read.parquet("/FileStore/tables/48se0cwp1497649233518/part_00000_cd03406a_cc9b_42b0_9299_1e259fdd9382_c000_gz-2e0be.parquet")

In [100]:
sales = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/FileStore/tables/1x42p66f1497639007910/*.csv")\
.coalesce(5)\
.where("Description IS NOT NULL")
fakeIntDF = spark.read.parquet("/FileStore/tables/w90rmm721497649161288/part_00000_ce2a44c8_feb4_4369_a2c3_4bf2f0e63b07_c000_gz-d65b9.parquet")
simpleDF = spark.read.json("/FileStore/tables/ixxwee3q1497648769294/part_r_00000_f5c243b9_a015_4a3b_a4a8_eca00f80f04c-a8b89.json")
scaleDF = spark.read.parquet("/FileStore/tables/48se0cwp1497649233518/part_00000_cd03406a_cc9b_42b0_9299_1e259fdd9382_c000_gz-2e0be.parquet")
sales.cache()

In [101]:
%scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer().setInputCol("Description")
tkn.transform(sales).show()

In [102]:
%scala
import org.apache.spark.ml.feature.StandardScaler
val ss = new StandardScaler().setInputCol("features")
ss.fit(scaleDF).transform(scaleDF).show(false)

In [103]:
%scala
import org.apache.spark.ml.feature.RFormula
val supervised = new RFormula()
.setFormula("lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

In [104]:
from pyspark.ml.feature import RFormula
supervised = RFormula()\
.setFormula("lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

In [105]:
%scala
import org.apache.spark.ml.feature.SQLTransformer
val basicTransformation = new SQLTransformer()
.setStatement("""
SELECT sum(Quantity), count(*), CustomerID
FROM __THIS__
GROUP BY CustomerID
""")
basicTransformation.transform(sales).show()

In [106]:
from pyspark.ml.feature import SQLTransformer
basicTransformation = SQLTransformer()\
.setStatement("""
SELECT sum(Quantity), count(*), CustomerID
FROM __THIS__
GROUP BY CustomerID
""")
basicTransformation.transform(sales).show()

In [107]:
%scala
import org.apache.spark.ml.feature.VectorAssembler
val va = new VectorAssembler()
.setInputCols(Array("int1", "int2", "int3"))
va.transform(fakeIntDF).show()

In [108]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

In [109]:
%scala
import org.apache.spark.ml.feature.Tokenizer
val tkn = new Tokenizer()
.setInputCol("Description")
.setOutputCol("DescriptionOut")
val tokenized = tkn.transform(sales)
tokenized.show()

In [110]:
from pyspark.ml.feature import Tokenizer
tkn = Tokenizer()\
.setInputCol("Description")\
.setOutputCol("DescriptionOut")
tokenized = tkn.transform(sales)
tokenized.show()

In [111]:
%scala
import org.apache.spark.ml.feature.RegexTokenizer
val rt = new RegexTokenizer()
.setInputCol("Description")
.setOutputCol("DescriptionOut")
.setPattern(" ") // starting simple
.setToLowercase(true)
rt.transform(sales).show()

In [112]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescriptionOut")\
.setPattern(" ")\
.setToLowercase(True)
rt.transform(sales).show()

In [113]:
%scala
import org.apache.spark.ml.feature.StopWordsRemover
val englishStopWords = StopWordsRemover
.loadDefaultStopWords("english")
val stops = new StopWordsRemover()
.setStopWords(englishStopWords)
.setInputCol("DescriptionOut")
stops.transform(tokenized).show()

In [114]:
from pyspark.ml.feature import StopWordsRemover
englishStopWords = StopWordsRemover\
.loadDefaultStopWords("english")
stops = StopWordsRemover()\
.setStopWords(englishStopWords)\
.setInputCol("DescriptionOut")
stops.transform(tokenized).show()

In [115]:
%scala
import org.apache.spark.ml.feature.NGram
val unigram = new NGram()
.setInputCol("DescriptionOut")
.setN(1)
val bigram = new NGram()
.setInputCol("DescriptionOut")
.setN(2)
unigram.transform(tokenized).show()
bigram.transform(tokenized).show()

In [116]:
%scala
import org.apache.spark.ml.feature.CountVectorizer
val cv = new CountVectorizer()
.setInputCol("DescriptionOut")
.setOutputCol("countVec")
.setVocabSize(500)
.setMinTF(1)
.setMinDF(2)
val fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show()

In [117]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer()\
.setInputCol("DescriptionOut")\
.setOutputCol("countVec")\
.setVocabSize(500)\
.setMinTF(1)\
.setMinDF(2)
fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show()

In [118]:
%scala
val tfIdfIn = tokenized
.where("array_contains(DescriptionOut, 'red')")
.select("DescriptionOut")
.limit(10)
tfIdfIn.show(false)

In [119]:
tfIdfIn = tokenized\
.where("array_contains(DescriptionOut, 'red')")\
.select("DescriptionOut")\
.limit(10)
tfIdfIn.show(10, False)

In [120]:
%scala
import org.apache.spark.ml.feature.{HashingTF, IDF}
val tf = new HashingTF()
.setInputCol("DescriptionOut")
.setOutputCol("TFOut")
.setNumFeatures(10000)
val idf = new IDF()
.setInputCol("TFOut")
.setOutputCol("IDFOut")
.setMinDocFreq(2)

In [121]:
from pyspark.ml.feature import HashingTF, IDF
tf = HashingTF()\
.setInputCol("DescriptionOut")\
.setOutputCol("TFOut")\
.setNumFeatures(10000)
idf = IDF()\
.setInputCol("TFOut")\
.setOutputCol("IDFOut")\
.setMinDocFreq(2)


In [122]:
%scala
idf.fit(tf.transform(tfIdfIn))
.transform(tf.transform(tfIdfIn))
.show(false)


In [123]:
idf.fit(tf.transform(tfIdfIn))\
.transform(tf.transform(tfIdfIn))\
.show(10, False)

In [124]:
%scala
val contDF = spark.range(500)
.selectExpr("cast(id as double)")

In [125]:
contDF = spark.range(500)\
.selectExpr("cast(id as double)")

In [126]:
%scala
import org.apache.spark.ml.feature.Bucketizer
val bucketBorders = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
val bucketer = new Bucketizer()
.setSplits(bucketBorders)
.setInputCol("id")
bucketer.transform(contDF).show()

In [127]:
from pyspark.ml.feature import Bucketizer
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer()\
.setSplits(bucketBorders)\
.setInputCol("id")
bucketer.transform(contDF).show()

In [128]:
%scala
import org.apache.spark.ml.feature.QuantileDiscretizer
val bucketer = new QuantileDiscretizer()
.setNumBuckets(5)
.setInputCol("id")
val fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

In [129]:
from pyspark.ml.feature import QuantileDiscretizer
bucketer = QuantileDiscretizer()\
.setNumBuckets(5)\
.setInputCol("id")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

In [130]:
%scala
import org.apache.spark.ml.feature.Normalizer
val taxicab = new Normalizer()
.setP(1)
.setInputCol("features")
taxicab.transform(scaleDF).show(false)

In [131]:
from pyspark.ml.feature import Normalizer
taxicab = Normalizer()\
.setP(1)\
.setInputCol("features")
taxicab.transform(scaleDF).show()

In [132]:
%scala
import org.apache.spark.ml.feature.StandardScaler
val sScaler = new StandardScaler()
.setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show(false)

In [133]:
%scala
import org.apache.spark.ml.feature.MinMaxScaler
val minMax = new MinMaxScaler()
.setMin(5)
.setMax(10)
.setInputCol("features")
val fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show(false)


In [134]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler()\
.setMin(5)\
.setMax(10)\
.setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

In [135]:
%scala
import org.apache.spark.ml.feature.MaxAbsScaler
val maScaler = new MaxAbsScaler()
.setInputCol("features")
val fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show(false)

In [136]:
%scala
import org.apache.spark.ml.feature.ElementwiseProduct
import org.apache.spark.ml.linalg.Vectors
val scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
val scalingUp = new ElementwiseProduct()
.setScalingVec(scaleUpVec)
.setInputCol("features")
scalingUp.transform(scaleDF).show()

In [137]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct()\
.setScalingVec(scaleUpVec)\
.setInputCol("features")
scalingUp.transform(scaleDF).show()

In [138]:
%scala
import org.apache.spark.ml.feature.StringIndexer
val labelIndexer = new StringIndexer()
.setInputCol("lab")
.setOutputCol("labelInd")
val idxRes = labelIndexer.fit(simpleDF).transform(simpleDF)
idxRes.show()

In [139]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer()\
.setInputCol("lab")\
.setOutputCol("labelInd")
idxRes = labelIndexer.fit(simpleDF).transform(simpleDF)
idxRes.show()

In [140]:
%scala
val valIndexer = new StringIndexer()
.setInputCol("value1")
.setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

In [141]:
valIndexer = StringIndexer()\
.setInputCol("value1")\
.setOutputCol("valueInd")
valIndexer.fit(simpleDF).transform(simpleDF).show()

In [142]:
%scala
valIndexer.setHandleInvalid("skip")
valIndexer.fit(simpleDF).setHandleInvalid("skip")

In [143]:
%scala
import org.apache.spark.ml.feature.IndexToString
val labelReverse = new IndexToString()
.setInputCol("labelInd")
labelReverse.transform(idxRes).show()

In [144]:
from pyspark.ml.feature import IndexToString
labelReverse = IndexToString()\
.setInputCol("labelInd")
labelReverse.transform(idxRes).show()

In [145]:
%scala
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.linalg.Vectors
val idxIn = spark.createDataFrame(Seq(
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)
)).toDF("features", "label")
val indxr = new VectorIndexer()
.setInputCol("features")
.setOutputCol("idxed")
.setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show

In [146]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
idxIn = spark.createDataFrame([
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)
]).toDF("features", "label")
indxr = VectorIndexer()\
.setInputCol("features")\
.setOutputCol("idxed")\
.setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show

In [147]:
%scala
val labelIndexer = new StringIndexer()
.setInputCol("color")
.setOutputCol("colorInd")
val colorLab = labelIndexer.fit(simpleDF).transform(simpleDF)

In [148]:
labelIndexer = StringIndexer()\
.setInputCol("color")\
.setOutputCol("colorInd")
colorLab = labelIndexer.fit(simpleDF).transform(simpleDF)

In [149]:
%scala
import org.apache.spark.ml.feature.OneHotEncoder
val ohe = new OneHotEncoder()
.setInputCol("colorInd")
ohe.transform(colorLab).show()

In [150]:
from pyspark.ml.feature import OneHotEncoder
ohe = OneHotEncoder()\
.setInputCol("colorInd")
ohe.transform(colorLab).show()

In [151]:
%scala
import org.apache.spark.ml.feature.PCA
val pca = new PCA()
.setInputCol("features")
.setK(2)
pca.fit(scaleDF).transform(scaleDF).show(false)

In [152]:
from pyspark.ml.feature import PCA
pca = PCA()\
.setInputCol("features")\
.setK(2)
pca.fit(scaleDF).transform(scaleDF).show()

In [153]:
%scala
import org.apache.spark.ml.feature.PolynomialExpansion
val pe = new PolynomialExpansion()
.setInputCol("features")
.setDegree(2)
pe.transform(scaleDF).show(false)

In [154]:
from pyspark.ml.feature import PolynomialExpansion
pe = PolynomialExpansion()\
.setInputCol("features")\
.setDegree(2)
pe.transform(scaleDF).show()

In [155]:
%scala
import org.apache.spark.ml.feature.ChiSqSelector
val prechi = fittedCV.transform(tokenized)
.where("CustomerId IS NOT NULL")
val chisq = new ChiSqSelector()
.setFeaturesCol("countVec")
.setLabelCol("CustomerID")
.setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi).show()

In [156]:
from pyspark.ml.feature import ChiSqSelector
prechi = fittedCV.transform(tokenized)\
.where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
.setFeaturesCol("countVec")\
.setLabelCol("CustomerID")\
.setNumTopFeatures(2)
chisq.fit(prechi).transform(prechi).show()

In [157]:
%scala
val fittedPCA = pca.fit(scaleDF)
fittedPCA.write.overwrite().save("/tmp/fittedPCA")

In [158]:
%scala
import org.apache.spark.ml.UnaryTransformer
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.types.{ArrayType, StringType, DataType}
import org.apache.spark.ml.param.{IntParam, ParamValidators}
class MyTokenizer(override val uid: String)
extends UnaryTransformer[String, Seq[String], MyTokenizer] with DefaultParamsWritable {
def this() = this(Identifiable.randomUID("myTokenizer"))
val maxWords: IntParam = new IntParam(this, "maxWords", "The max number of words to return.",
ParamValidators.gtEq(0))
def setMaxWords(value: Int): this.type = set(maxWords, value)
  def getMaxWords: Integer = $(maxWords)
override protected def createTransformFunc: String => Seq[String] = (inputString: String) => {
inputString.split("\\s").take($(maxWords))
}
override protected def validateInputType(inputType: DataType): Unit = {
require(inputType == StringType, s"Bad input type: $inputType. Requires String.")
}
override protected def outputDataType: DataType = new ArrayType(StringType, true)
}
// this will allow you to read it back in by using this object.
object MyTokenizer extends DefaultParamsReadable[MyTokenizer]
val myT = new MyTokenizer()
.setInputCol("someCol")
.setMaxWords(2)
display(myT.transform(Seq("hello world. This text won't show.").toDF("someCol")))
myT.write.overwrite().save("/tmp/something")